### Rastreo de noticias en Google News sobre temas de la Ciudad de México que competan o sean de interés del Instituto de Planeación de la Ciudad de México

El script extrae las notas de Google News de los temas de agua, movilidad, planeación, salud y seguridad de la Ciudad de México. El script extrae los títulos, url y fecha de las notas.

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
#Temas de interés

temas=["agua","movilidad","planeacion","salud","seguridad"]

In [32]:
#Función para extraer notas:
def get_notas(tema,periodo):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
    }
    url = f'https://news.google.com/search?q={tema}%20ciudad%20de%20mexico%20when%3A{periodo}d&hl=es-419&gl=MX&ceid=MX%3Aes-419'
    response = requests.get(url, headers=headers)
    sopa=BeautifulSoup(response.text, 'html.parser')
    resultados=sopa.find_all('article')
    #Extraer títulos y url
    titulos=[]
    url=[]
    fechas=[]
    for i in range(len(resultados)):
        titulos.append(resultados[i].find('a', class_='JtKRv').text)
        url.append(resultados[i].find('a').get('href'))
        fecha = resultados[i].find('time')['datetime'] if resultados[i].find('time') else None
        fechas.append(fecha)
    # Eliminar el primer punto de la url
    url = [i[1:] for i in url]

    df=pd.DataFrame({'titulo':titulos,'url':url,'fecha_nota':fechas,'fecha_consulta':pd.to_datetime('today').strftime('%Y-%m-%d'),'tema':tema})
    df['url'] = 'https://news.google.com' + df['url']

    return df

In [33]:
#Tabla general de notas
consolidado_notas=pd.DataFrame()

#Ciclo para extraer las notas de ayer de los temas
for tema in temas:
    notas=get_notas(tema,1)
    consolidado_notas=pd.concat([consolidado_notas,notas],axis=0)


In [34]:
consolidado_notas

,titulo,url,fecha_nota,fecha_consulta,tema
0,¿Cómo pagar un recibo de agua vencido en la CDMX?,https://news.google.com/read/CBMioAFBVV95cUxOR...,2025-03-13T18:58:16Z,2025-03-13,agua
1,Sequía en México; ¿Cuáles son los riesgos de a...,https://news.google.com/read/CBMi1AFBVV95cUxPU...,2025-03-13T17:08:35Z,2025-03-13,agua
2,Conagua invertirá 195 mdp para rehabilitar dre...,https://news.google.com/read/CBMiqAFBVV95cUxNc...,2025-03-13T11:08:40Z,2025-03-13,agua
3,¿Corte de agua en CDMX?: Estas son las alcaldí...,https://news.google.com/read/CBMitAFBVV95cUxPd...,2025-03-12T21:25:51Z,2025-03-13,agua
4,Agua para el futuro: el reto de México,https://news.google.com/read/CBMinwFBVV95cUxOa...,2025-03-13T06:13:00Z,2025-03-13,agua
...,...,...,...,...,...
50,De cuánto es la MULTA por tapar números o letr...,https://news.google.com/read/CBMinAFBVV95cUxNN...,2025-03-12T23:29:00Z,2025-03-13,seguridad
51,"En Tláhuac, mototaxista sin una pierna andaba ...",https://news.google.com/read/CBMi3gFBVV95cUxPd...,2025-03-12T22:36:00Z,2025-03-13,seguridad
52,"Detención de El Conejo, presunto líder de Los ...",https://news.google.com/read/CBMixgFBVV95cUxQd...,2025-03-12T20:57:00Z,2025-03-13,seguridad
53,"En Tláhuac, mototaxista sin una pierna andaba ...",https://news.google.com/read/CBMi2gFBVV95cUxOd...,2025-03-12T22:53:16Z,2025-03-13,seguridad
